In [1]:
import SAGA
from SAGA.utils import add_skycoord
from SAGA.objects.build import *
from SAGA.database import FitsTable

print('SAGA code version', SAGA.__version__)

SAGA code version 0.4.0


In [2]:
import os
import time

In [3]:
saga_database = SAGA.Database('/home/yymao/Dropbox/Academia/Collaborations/SAGA')
saga_database.wise_file_path_pattern = '/home/yymao/Documents/Research/SAGA/external_catalogs/wise/nsa{}.fits.gz'
saga_database['spectra_gama'].local = '/home/yymao/Documents/Research/SAGA/external_catalogs/GAMA_SpecObj.fits'
saga_database['nsa_v0.1.2'].local = '/home/yymao/Documents/Research/SAGA/external_catalogs/nsa_v0_1_2.fits'

In [4]:
saga_host_catalog = SAGA.HostCatalog(saga_database)

In [5]:
nsa = add_skycoord(saga_database['nsa_v0.1.2'].read()[nsa_cols_used])

In [6]:
spectra_raw_all = saga_database['spectra_raw_all'].read()

In [7]:
SDSS_DR = 12
saga_database.sdss_file_path_pattern = '/home/yymao/Documents/Research/SAGA/external_catalogs/sdss_dr{}/nsa{{}}.fits.gz'.format(SDSS_DR)
base_output_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr{}_nsa{{}}.fits.gz'.format(SDSS_DR)
print('Using DR{}'.format(SDSS_DR))

overwrite = True
host_ids = saga_host_catalog.resolve_id('paper1')

for i, host_id in enumerate(host_ids):
    output_path = base_output_pattern.format(host_id)
    if os.path.isfile(output_path) and not overwrite:
        continue
    
    print(time.strftime('[%m/%d %H:%M:%S]'), 'Working on NSA', host_id, '({}/{})'.format(i+1, len(host_ids)))
    
    try:
        wise = saga_database['wise', host_id].read()[wise_cols_used]
    except OSError:
        wise = None
        
    base = build_full_stack(saga_database['sdss', host_id].read(), saga_host_catalog.load_single(host_id), 
                            saga_database['hosts_named'].read(), wise, nsa, 
                            saga_database['objects_to_remove'].read(), saga_database['objects_to_add'].read(), 
                            spectra_raw_all)

    FitsTable(output_path).write(base)

Using DR12
[09/01 23:36:03] Working on NSA 166313 (1/16)


 Too many (> 25) shreds around SDSS 1237668367458828369 (233.98344668799618, 16.070882039023196)


[09/01 23:37:50] Working on NSA 147100 (2/16)
[09/01 23:39:52] Working on NSA 165536 (3/16)


 No object within the radius of NSA 145531 (221.0511483286393, 0.17499166664364407)


[09/01 23:41:49] Working on NSA 61945 (4/16)


 No object can be matched to MMT spec (354.20142174, 0.498705)
 No object can be matched to MMT spec (354.199333185, 0.295363009)
 No object can be matched to MMT spec (354.13258552499997, 0.261272997)
 No object can be matched to MMT spec (354.05522346, 0.280936003)


[09/01 23:43:08] Working on NSA 132339 (5/16)
[09/01 23:44:08] Working on NSA 149781 (6/16)
[09/01 23:44:55] Working on NSA 33446 (7/16)
[09/01 23:45:50] Working on NSA 150887 (8/16)
[09/01 23:46:36] Working on NSA 161174 (9/16)
[09/01 23:47:11] Working on NSA 85746 (10/16)
[09/01 23:47:57] Working on NSA 145729 (11/16)
[09/01 23:48:32] Working on NSA 140594 (12/16)
[09/01 23:49:47] Working on NSA 126115 (13/16)
[09/01 23:50:08] Working on NSA 13927 (14/16)
[09/01 23:50:54] Working on NSA 137625 (15/16)
[09/01 23:51:17] Working on NSA 129237 (16/16)


## Some docstring

In [9]:
print(build_full_stack.__doc__)


    This function calls all needed functions to complete the full stack of building
    a base catalog (for a single host), in the following order:

    >>> initialize_base_catalog(base)
    >>> add_host_info(base, host, saga_names)
    >>> add_wise(base, wise)
    >>> remove_human_inspected(base, objects_to_remove)
    >>> remove_too_close_to_host(base)
    >>> remove_shreds_with_nsa(base, nsa)
    >>> remove_shreds_with_sdss(base)
    >>> remove_bad_photometry(base)
    >>> recover_whitelisted_objects(base, objects_to_add)
    >>> apply_manual_fixes(base)
    >>> add_spectra(base, spectra)
    >>> clean_sdss_spectra(base)
    >>> find_satellites(base)
    >>> add_stellar_mass(base)

    See docstring of each function to learn more.

    `base` is always modified in-place.

    Parameters
    ----------
    base : astropy.table.Table
    host : astropy.table.Row
    saga_names : astropy.table.Table
    wise : astropy.table.Table
    nsa : astropy.table.Table
    objects_to_remove : a

In [14]:
print(add_cleaned_spectra.__doc__)


    Add cleaned spectra to base catalog.
    For each entry in `spectra_clean`, search nearby objects in base catalog
    to find an object to match the spectrum to.

    The search is done in the following order.
    In each category, if any objects are found, the closest one (on the sky)
    will be the match, and the search ends.

    1. within 20 arcsec, spec within +/- 50 km/s, REMOVE==-1, has NSA
    2. within 20 arcsec, spec within +/- 50 km/s, REMOVE==-1, any ZQUALITY==4
    3. within 3 arcsec, REMOVE==-1
    4. within 3 arcsec

    Once a match is found, any other objects that satisfy the condition (2) above
    are turned off (REMOVE set to 3).

    `base` is modified in-place.
    `spectra_clean` is expected to be the output of `clean_repeat_spectra`

    Parameters
    ----------
    base : astropy.table.Table
    spectra_clean : astropy.table.Table

    Returns
    -------
    base : astropy.table.Table

    Notes
    -----
    `add_spectra` calls this function.

    
